In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from matplotlib import pyplot as plt
import glob
from imdb import IMDb
ia = IMDb()
from time import sleep

In [2]:
netflix_file_path = 'netflix_titles.csv'
netflix_df = pd.read_csv(netflix_file_path)
netflix_df["Platform"] = "Netflix"

hulu_file_path = 'hulu_titles.csv'
hulu_df = pd.read_csv(hulu_file_path)
hulu_df["Platform"] = "Hulu"

disney_file_path = 'disney_plus_titles.csv'
disney_df = pd.read_csv(disney_file_path)
disney_df["Platform"] = "Disney"

amazon_prime_file_path = 'amazon_prime_titles.csv'
amazon_df = pd.read_csv(amazon_prime_file_path)
amazon_df["Platform"] = "Amazon"

In [3]:
df = pd.concat([netflix_df, hulu_df, amazon_df, disney_df], ignore_index=True)
df.drop_duplicates(inplace=True)
df.drop(columns=["cast"], inplace=True)
df.drop(columns=["country"], inplace=True)
df.drop(columns=["description"], inplace=True)
df["Rating"] = 0
display(df)

,show_id,type,title,director,date_added,release_year,rating,duration,listed_in,Platform,Rating
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,"September 25, 2021",2020,PG-13,90 min,Documentaries,Netflix,0
1,s2,TV Show,Blood & Water,NaN,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries",Netflix,0
2,s3,TV Show,Ganglands,Julien Leclercq,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",Netflix,0
3,s4,TV Show,Jailbirds New Orleans,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV",Netflix,0
4,s5,TV Show,Kota Factory,NaN,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",Netflix,0
...,...,...,...,...,...,...,...,...,...,...,...
22993,s1446,Movie,X-Men Origins: Wolverine,Gavin Hood,"June 4, 2021",2009,PG-13,108 min,"Action-Adventure, Family, Science Fiction",Disney,0
22994,s1447,Movie,Night at the Museum: Battle of the Smithsonian,Shawn Levy,"April 2, 2021",2009,PG,106 min,"Action-Adventure, Comedy, Family",Disney,0
22995,s1448,Movie,Eddie the Eagle,Dexter Fletcher,"December 18, 2020",2016,PG-13,107 min,"Biographical, Comedy, Drama",Disney,0
22996,s1449,Movie,Bend It Like Beckham,Gurinder Chadha,"September 18, 2020",2003,PG-13,112 min,"Buddy, Comedy, Coming of Age",Disney,0


In [ ]:
def get_imdb_rating(title, year=None):
    try:
        results = ia.search_movie(title)
        if year:
            # Try to find a closer match using the year
            results = [m for m in results if 'year' in m and m['year'] == year]
        if not results:
            return None
        movie = ia.get_movie(results[0].movieID)
        return movie.get('rating', None)
    except:
        return None

ratings = []
# Iterate through the DataFrame and fetch IMDb ratings starting from start_row
for index, row in df[10000:].iterrows():
    title = row['title']
    year = row['year'] if 'year' in df.columns else None
    rating = get_imdb_rating(title, year)
    ratings.append(rating)
    print(f"{index+1}/{len(df)}: {title} - {rating}")

#turn ratings into a csv
ratings_df = pd.DataFrame({'title': df['title'][10000:], 'imdb_rating': ratings})
ratings_df.to_csv('imdb_ratings.csv', index=False)


1/22998: Dick Johnson Is Dead - 7.4
2/22998: Blood & Water - 6.7
3/22998: Ganglands - 7.2
4/22998: Jailbirds New Orleans - 6.5
5/22998: Kota Factory - 9.0
6/22998: Midnight Mass - 7.7
7/22998: My Little Pony: A New Generation - 6.8
8/22998: Sankofa - 7.0
9/22998: The Great British Baking Show - 8.1
10/22998: The Starling - 6.4
11/22998: Vendetta: Truth, Lies and The Mafia - 6.7
12/22998: Bangkok Breaking - 5.8
13/22998: Je Suis Karl - 5.6
14/22998: Confessions of an Invisible Girl - 5.3
15/22998: Crime Stories: India Detectives - 7.0
16/22998: Dear White People - 6.4
17/22998: Europe's Most Dangerous Man: Otto Skorzeny in Spain - 6.8
18/22998: Falsa identidad - 7.0
19/22998: Intrusion - 5.3
20/22998: Jaguar - 5.8
21/22998: Monsters Inside: The 24 Faces of Billy Milligan - 6.3
22/22998: Resurrection: Ertugrul - 7.9
23/22998: Avvai Shanmughi - 7.9
24/22998: Go! Go! Cory Carson: Chrissy Takes the Wheel - 6.6
25/22998: Jeans - 8.0
26/22998: Love on the Spectrum - 8.4
27/22998: Minsara Kana